In [1]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# specify the directory containing the CSV files
directory = "C://Users//20212397//Desktop//Y2Q4//DC2//"

# initialize an empty list to store dataframes
df_list = []

# iterate through all subfolders and CSV files in the directory
for root, dirs, files in os.walk(directory):
    for filename in files:
        # print(filename)
        #Change csv names
        if filename.endswith("metropolitan-street.csv"): 
            # print(filename)
            # read the CSV file into a dataframe and append to the list
            filepath = os.path.join(root, filename)
            df = pd.read_csv(filepath)
            df_list.append(df)

# concatenate all dataframes into a single dataframe
concatenated_df = pd.concat(df_list, axis=0, ignore_index=True)
# write the concatenated dataframe to a new parquet file
output_filepath = "C://Users//20212397//Desktop//Y2Q4//DC2//Metropolitan-street.parquet"

# write the concatenated dataframe to a Parquet file
table = pa.Table.from_pandas(concatenated_df)
pq.write_table(table, output_filepath)

In [2]:
pip install fsspec

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\20212397\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# Read the Parquet file into a pandas DataFrame
df = pd.read_parquet('C://Users//20212397//Desktop//Y2Q4//DC2//Metropolitan-street.parquet')

# Do something with the DataFrame
df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.145949,51.593914,On or near Providence Place,E01000027,Barking and Dagenham 001A,Anti-social behaviour,None,NaN
1,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.140192,51.582311,On or near Hatch Grove,E01000027,Barking and Dagenham 001A,Anti-social behaviour,None,NaN
2,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.134947,51.588063,On or near Mead Grove,E01000027,Barking and Dagenham 001A,Anti-social behaviour,None,NaN
3,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.134947,51.588063,On or near Mead Grove,E01000027,Barking and Dagenham 001A,Anti-social behaviour,None,NaN
4,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.134947,51.588063,On or near Mead Grove,E01000027,Barking and Dagenham 001A,Anti-social behaviour,None,NaN


In [4]:
# Filter the DataFrame to only include rows where the LSOA name is 'Barnet'
filtered_df = df[df['LSOA name'].str.contains('Barnet') == True]
filtered_df = df[df['Crime type'].str.contains('Burglary') == True]

# Do something with the filtered DataFrame
filtered_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
29,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.140619,51.583427,On or near Rams Grove,E01000027,Barking and Dagenham 001A,Burglary,None,NaN
47,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.128479,51.583414,On or near Cavalier Close,E01000028,Barking and Dagenham 001B,Burglary,None,NaN
60,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.134801,51.578470,On or near Portland Close,E01000029,Barking and Dagenham 001C,Burglary,None,NaN
61,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.131156,51.582323,On or near Lake Road,E01000029,Barking and Dagenham 001C,Burglary,None,NaN
84,None,2010-12,Metropolitan Police Service,Metropolitan Police Service,0.140466,51.584159,On or near Bagleys Spring,E01000030,Barking and Dagenham 001D,Burglary,None,NaN


In [5]:
# initialize an empty list to store dataframes
df_list2 = []

# iterate through all subfolders and CSV files in the directory
for root, dirs, files in os.walk(directory):
    for filename in files:
        # print(filename)
        #Change csv names
        if filename.endswith("metropolitan-stop-and-search.csv"): 
            # print(filename)
            # read the CSV file into a dataframe and append to the list
            filepath = os.path.join(root, filename)
            df2 = pd.read_csv(filepath)
            df_list2.append(df2)

# concatenate all dataframes into a single dataframe
concatenated_df2 = pd.concat(df_list2, axis=0, ignore_index=True)
# write the concatenated dataframe to a new parquet file
output_filepath = "C://Users//20212397//Desktop//Y2Q4//DC2//Metropolitan-stop-and-search.parquet"

# write the concatenated dataframe to a Parquet file
table2 = pa.Table.from_pandas(concatenated_df2)
pq.write_table(table2, output_filepath)

C:\Users\20212397\AppData\Local\Temp\ipykernel_23988\1370394234.py:17: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  concatenated_df2 = pd.concat(df_list2, axis=0, ignore_index=True)


In [6]:
concatenated_df2.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
0,Person and Vehicle search,2015-03-31T23:00:00+00:00,NaN,NaN,NaN,NaN,Male,18-24,White - White British (W1),White,Police and Criminal Evidence Act 1984 (section 1),NaN,Nothing found - no further action,NaN,NaN
1,Person and Vehicle search,2015-03-31T23:00:00+00:00,NaN,NaN,NaN,NaN,Male,18-24,White - White British (W1),White,Police and Criminal Evidence Act 1984 (section 1),NaN,Suspect arrested,NaN,NaN
2,Person search,2015-03-31T23:05:00+00:00,NaN,NaN,NaN,NaN,Male,18-24,Black or Black British - Any other Black ethni...,Black,Misuse of Drugs Act 1971 (section 23),NaN,Offender given drugs possession warning,NaN,NaN
3,Person search,2015-03-31T23:09:00+00:00,NaN,NaN,NaN,NaN,Male,NaN,Not Stated (NS),Black,Misuse of Drugs Act 1971 (section 23),NaN,Nothing found - no further action,NaN,NaN
4,Person search,2015-03-31T23:15:00+00:00,NaN,NaN,NaN,NaN,Male,18-24,Asian or Asian British - Any other Asian ethni...,Asian,Police and Criminal Evidence Act 1984 (section 1),NaN,Nothing found - no further action,NaN,NaN


In [7]:
# initialize an empty list to store dataframes
df_list3 = []

# iterate through all subfolders and CSV files in the directory
for root, dirs, files in os.walk(directory):
    for filename in files:
        # print(filename)
        #Change csv names
        if filename.endswith("metropolitan-outcomes.csv"): 
            # print(filename)
            # read the CSV file into a dataframe and append to the list
            filepath = os.path.join(root, filename)
            df3 = pd.read_csv(filepath)
            df_list3.append(df3)

# concatenate all dataframes into a single dataframe
concatenated_df3 = pd.concat(df_list3, axis=0, ignore_index=True)
# write the concatenated dataframe to a new parquet file
output_filepath = "C://Users//20212397//Desktop//Y2Q4//DC2//Metropolitan-outcomes.parquet"

# write the concatenated dataframe to a Parquet file
table3 = pa.Table.from_pandas(concatenated_df3)
pq.write_table(table3, output_filepath)

In [8]:
# Filter the DataFrame to only include rows where the LSOA name is 'Barnet'
filtered_df3 = df3[df3['LSOA name'].str.contains('Barnet') == True]

# Do something with the filtered DataFrame
filtered_df3.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Outcome type
259,e73755559741d7251e0819e9e29b98b400dcf6365ab1fe...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.256798,51.639353,On or near,E01000228,Barnet 007A,Investigation complete; no suspect identified
368,1ff548bd410402370a3d7de9d1ade30f0250c2989b6d5e...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.202006,51.598547,On or near FRESTON PARK,E01000200,Barnet 028B,Investigation complete; no suspect identified
689,0d22d58852e291bcf6c99a63963f02dc9de2d0692931fd...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.159526,51.612143,On or near,E01000162,Barnet 015E,Suspect charged
1182,181fff2d5861d20ed728e9f6ba033f32978433285935b7...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.214148,51.650727,On or near,E01000294,Barnet 004F,Investigation complete; no suspect identified
1230,f9bee37a68797be96f89f78acae46225eb5a1b32873de9...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.151616,51.620911,On or near Link Road,E01000158,Barnet 015A,Investigation complete; no suspect identified


In [11]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\20212397\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import requests

In [38]:
locations = [
    "High Barnet",
    "Underhill",
    "Barnet Vale",
    "East Barnet",
    "Friern Barnet",
    "Woodhouse",
    "Whetstone",
    "Brunswick Park",
    "Totteridge and Woodside",
    "Mill Hill",
    "Cricklewood",
    "Edgwarebury",
    "Burnt Oak",
    "Edgware",
    "Colindale South",
    "West Hendon",
    "Colindale North",
    "Hendon",
    "West Finchley",
    "East Finchley",
    "Garden Suburb",
    "Finchley Church End",
    "Golders Green",
    "Childs Hill"
]

In [39]:
response = requests.get("https://data.police.uk/api/metropolitan/neighbourhoods")
print(response.status_code)

200


In [40]:
print(response.json())

[{'id': 'E05009317', 'name': 'Bethnal Green East'}, {'id': 'E05009318', 'name': 'Blackwall and Cubitt Town'}, {'id': 'E05009319', 'name': 'Bow East'}, {'id': 'E05009320', 'name': 'Bow West'}, {'id': 'E05009321', 'name': 'Bromley North'}, {'id': 'E05009322', 'name': 'Bromley South'}, {'id': 'E05009323', 'name': 'Canary Wharf'}, {'id': 'E05009324', 'name': 'Island Gardens'}, {'id': 'E05009325', 'name': 'Lansbury'}, {'id': 'E05009326', 'name': 'Limehouse'}, {'id': 'E05009327', 'name': 'Mile End'}, {'id': 'E05009328', 'name': 'Poplar'}, {'id': 'E05009329', 'name': "St Dunstan's"}, {'id': 'E05009330', 'name': "St Katharine's and Wapping"}, {'id': 'E05009331', 'name': 'Bethnal Green West'}, {'id': 'E05009332', 'name': 'Shadwell'}, {'id': 'E05009333', 'name': 'Spitalfields and Banglatown'}, {'id': 'E05009334', 'name': 'Stepney Green'}, {'id': 'E05009335', 'name': 'Weavers'}, {'id': 'E05009336', 'name': 'Whitechapel'}, {'id': 'E05009367', 'name': 'Brownswood'}, {'id': 'E05009368', 'name': 'Caz

In [43]:
barnet_ids = []

for d in response.json():
    if d['name'] in locations:
        barnet_ids.append(d["id"])

In [44]:
print(barnet_ids)
len(barnet_ids)

['E05013545', 'E05013628', 'E05013629', 'E05013630', 'E05013631', 'E05013632', 'E05013633', 'E05013634', 'E05013635', 'E05013636', 'E05013637', 'E05013638', 'E05013639', 'E05013640', 'E05013641', 'E05013642', 'E05013643', 'E05013644', 'E05013645', 'E05013646', 'E05013647', 'E05013648', 'E05013649', 'E05013650', 'E05013651']


25

In [13]:
# DO SOMETHING WITH BARNET WARDS AND VISUALIZE THEM IN A MAP
# NOT VALID CODE

st.title("Scatter mapbox")
px.set_mapbox_access_token("pk.eyJ1IjoibWl0a28yMDAxIiwiYSI6ImNsZGtoYzdrMzAxdnkzcm1pOHVlaGYxaGoifQ.7ANpE95HzAWjZzuzFEMSTQ")
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="LSOA code", hover_data=['neighbourhood_group','neighbourhood','room_type','price', 'service_fee'],
                        color='neighbourhood_group', zoom=8, height=300)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
st.plotly_chart(fig, use_container_width=True)

NameError: name 'st' is not defined

In [9]:
pip install geopandas matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\20212397\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
#NEED INFO FOR BARNET WARD BOUNDARIES
#NOT VALID CODE

import geopandas as gpd
import matplotlib.pyplot as plt

# Load the wards boundary data
wards_url = 'YOUR_WARDS_BOUNDARY_URL'  # Replace this with the URL of your wards boundary file
wards = gpd.read_file(wards_url)

# Sample list of burglary coordinates, replace with your own data
# (longitude, latitude) format
burglary_coords = [
    (-0.196503, 51.614696),
    (-0.164635, 51.629993),
    (-0.214981, 51.624074),
    (-0.183965, 51.606903),
]

# Convert the list of coordinates to a GeoDataFrame
burglary_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy([x[0] for x in burglary_coords], [x[1] for x in burglary_coords]))

# Set the same Coordinate Reference System (CRS) as the wards data
burglary_gdf.crs = wards.crs

# Plot the map with the burglary points
fig, ax = plt.subplots(figsize=(10, 10))
wards.boundary.plot(ax=ax, color='black', linewidth=1)
burglary_gdf.plot(ax=ax, marker='o', color='red', markersize=50)
ax.set_title('Burglary Crimes in Barnet LSOA')
plt.show()

DriverError: YOUR_WARDS_BOUNDARY_URL: No such file or directory